<a href="https://colab.research.google.com/github/rdsiese/MANDES/blob/main/4_Barrier_options.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Barrier Options

The payoff of a path-dependent option depends on the evolution of the underlying’s price during the life of the option, not just at maturity $𝑇$.

In a barrier option, the payoff depends on whether a given price level (**barrier**) was reached or not by the underlying before maturity.

A **knock-in option** can be exercised if the asset price reached the barrier before $𝑇$.

A **knock-out option** can be exercised if the asset price never reached the barrier before $𝑇$.

Knock-out options can be down-and-out or up-and-out.

The payoff of an **up-and-out** option is $\max(S_t-K,0)$ if the underlying's price never reaches the barrier during the option's life; otherwise, the payoff is $0$.

In [ ]:
import numpy as np

S0=100; r=0.05; sigma=0.2; T=1

To get an idea of how the code works, we will initially divide the option's life in 4 steps, and only consider 3 paths of the underlying's price. We first set up the "containers" that will hold the values for the Brownian motion ($\texttt{brownian}$), sthe stock price ($\texttt{S}$) and the payoff ($\texttt{payoff}$). We set the initial value of the stock price to be $S_0$.

In [ ]:
brownian=np.zeros([3,5])
S=np.zeros([3,5])
payoff=np.zeros(3)

S[:,0]=S0

The following command generates the samples from the Normal random variables for each path. Execute the code several times to see how the values o z change.  

In [ ]:
z=np.random.normal(0,1,[3,4])
z

The following code computes 3 paths of the underlying. A more streamlined version could be written defining a function. The code below is meant to facilitate the class presentation.   

In [ ]:
brownian=np.zeros([3,5])
S=np.zeros([3,5])
payoff=np.zeros(3)

S[:,0]=S0
z=np.random.normal(0,1,[3,4])

for i in range(0,3):
    for j in range(1, 5):
        brownian[i,j]=brownian[i,j-1] + np.sqrt(T/4)*z[i,j-1]
        S[i,j] = S0*np.exp((r-0.5*sigma**2)*(j*T/4)+sigma*brownian[i,j])

S.round(4)

In [ ]:
barrier = 110; K = 80
z=np.random.normal(0,1,[3,4])

for i in range(0,3):
    for j in range(1, 5):
        brownian[i,j]=brownian[i,j-1] + np.sqrt(T/4)*z[i,j-1]
        S[i,j] = S0*np.exp((r-0.5*sigma**2)*(j*T/4)+sigma*brownian[i,j])

        if np.max(S[i,:]) > barrier:  # Check if barrier was reached in path
            payoff[i] = 0
        else:
            payoff[i] = np.maximum(S[i,4]-K, 0)

print(f'S:\n{S.round(4)} \n\nPayoff: {payoff.round(4)}')

We now divide the option's life in 300 steps and consider $1,000$ paths.

We only need to add two new lines to compute: (a) the discounted value of the mean payoff, and (b) the mean standard error.

In [ ]:
steps = 300; n = 1000; barrier = 110; K = 90

z=np.random.normal(0,1,[n,steps])
brownian=np.zeros([n,steps+1])
S=np.zeros([n,steps+1])
payoff=np.zeros(n)

S[:,0]=S0

for i in range(0,n):
    for j in range(1, steps+1):
        brownian[i,j]=brownian[i,j-1] + np.sqrt(T/steps)*z[i,j-1]
        S[i,j] = S0*np.exp((r-0.5*sigma**2)*(j*T/steps)+sigma*brownian[i,j])
    if np.max(S[i,:]) > barrier:                             # Check if barrier was reached in path
        payoff[i] = 0
    else:
        payoff[i] = np.maximum(S[i,steps]-K, 0)

KOprice = np.exp(-r*T)*np.mean(payoff)
mse = np.std(payoff) / np.sqrt(n)

print(f'price: {KOprice:<12.5f} mse: {mse:<10.3f} barrier: {barrier:<7} mse/price: {mse/KOprice:.3f}')



We finally extend the code to compute the barrier option's price for different values of the barrier.

Being an **up-and-out** option, the higher the barrier, the closer is price will be to a vanilla call.   
  

In [ ]:
steps = 300; n = 1000; K = 90

for barrier in[110, 120, 140, 160, 180, 200]:
    z=np.random.normal(0,1,[n,steps])
    brownian=np.zeros([n,steps+1])
    S=np.zeros([n,steps+1])
    payoff=np.zeros(n)

    S[:,0]=S0

    for i in range(0,n):
        for j in range(1, steps+1):
            brownian[i,j]=brownian[i,j-1] + np.sqrt(T/steps)*z[i,j-1]
            S[i,j] = S0*np.exp((r-0.5*sigma**2)*(j*T/steps)+sigma*brownian[i,j])
        if np.max(S[i,:]) > barrier:  # Check if barrier was reached in path
            payoff[i] = 0
        else:
            payoff[i] = np.maximum(S[i,steps]-K, 0)

    KOprice = np.exp(-r*T)*np.mean(payoff)
    mse = np.std(payoff) / np.sqrt(n)

    print(f'price: {KOprice:<12.5f} mse: {mse:<10.3f} b: {barrier:<7} mse/price: {mse/KOprice:.3f}')

The MSE is high, because the sample size is low ($n=1,000$). Execute the code again with $n=10,000$ and $n=20,000$.  

By changing the payoff, the above method can be applied to other path-dependent options, like lookback or Asian options.